In [29]:
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth
from IPython.display import HTML

## Data processing

#### How to read the data

Lecture : Compte tenu de la structure par âge de la population, les habitants de l'Abergement-Clémenciat ont accès en moyenne à 2,2 consultations ou visites de médecine générale par an et par habitant. 
Ce chiffre s'élève à 2,1 quand on restreint l'offre de médecine générale à celle fournie par les médecins de 65 ans ou moins.			

Note : Pour calculer un APL moyen (agrégé au niveau du territoire de vie-santé ou du département par exemple), l'APL communal doit être pondéré par la population standardisée. 

Pour comptabiliser la population sous un seuil d'APL ou calculer des quantiles, la population non standardisée doit être utilisée.		

#### Read apl data

In [30]:
def read_apl_csv(year):
    apl = pd.read_excel(r'C:\Users\idris\OneDrive\Documents\Study\M2\data vis\apl.xlsx', sheet_name='APL_' + str(year), skiprows=8)
    apl = apl.drop(0)
    apl['Code commune INSEE'] = pd.to_numeric(apl['Code commune INSEE'], errors='coerce')
    return apl

In [31]:
apl_2015=read_apl_csv(2015)
apl_2016=read_apl_csv(2016)
apl_2017=read_apl_csv(2017)
apl_2018=read_apl_csv(2018)
apl_2019=read_apl_csv(2019)
apl_2021=read_apl_csv(2021)

In [32]:
apl_2019.head()

,Code commune INSEE,Commune,APL aux médecins généralistes,APL aux médecins généralistes de moins de 65 ans,Population standardisée 2017 pour la médecine générale,Population totale 2017
1,1001.0,L'Abergement-Clémenciat,2.598,2.289,770.385,776
2,1002.0,L'Abergement-de-Varey,2.888,2.802,246.083,248
3,1004.0,Ambérieu-en-Bugey,4.569,4.469,13676.946,14035
4,1005.0,Ambérieux-en-Dombes,4.19,4.137,1651.149,1689
5,1006.0,Ambléon,0.861,0.775,116.576,111


In [33]:
apl_2021.tail()

,Code commune INSEE,Commune,APL aux médecins généralistes,APL aux médecins généralistes de moins de 65 ans,Population standardisée 2019 pour la médecine générale,Population totale 2019
34970,75116.0,Paris 16e Arrondissement,3.546,2.768,167614.21,165523
34971,75117.0,Paris 17e Arrondissement,4.662,3.752,159595.107,166543
34972,75118.0,Paris 18e Arrondissement,4.267,3.479,180410.01,192468
34973,75119.0,Paris 19e Arrondissement,4.672,3.843,176730.213,184573
34974,75120.0,Paris 20e Arrondissement,4.17,3.404,187719.246,194994


#### communes-departement-region

Ajout des regions et des departement
On realisera une joiture entre 2 fichiers  csv:
 - APL
 - communes-departement-region

In [34]:
cdp = pd.read_csv(r'C:\Users\idris\OneDrive\Documents\Study\M2\data vis\communes-departement-region.csv')

In [35]:
cdp.columns

Index(['code_commune_INSEE', 'nom_commune_postal', 'code_postal',
       'libelle_acheminement', 'ligne_5', 'latitude', 'longitude',
       'code_commune', 'article', 'nom_commune', 'nom_commune_complet',
       'code_departement', 'nom_departement', 'code_region', 'nom_region'],
      dtype='object')

In [36]:
cdp = cdp.rename(columns={'code_commune_INSEE': 'Code commune INSEE'})

In [37]:
cdp['Code commune INSEE'] = pd.to_numeric(cdp['Code commune INSEE'], errors='coerce')

In [38]:
cdp.head()

,Code commune INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001.0,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002.0,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004.0,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005.0,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006.0,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes


#### Merge apl & region-departement-commune

In [39]:
def merge_apl_cdp(apl):
    return pd.merge(apl, cdp, on='Code commune INSEE')

In [40]:
# join
apl_2015 = merge_apl_cdp(apl_2015)
apl_2016 = merge_apl_cdp(apl_2016)
apl_2017 = merge_apl_cdp(apl_2017)
apl_2018 = merge_apl_cdp(apl_2018)
apl_2019 = merge_apl_cdp(apl_2019)
apl_2021 = merge_apl_cdp(apl_2021)

In [41]:
apl_2015.head()

,Code commune INSEE,Commune,APL aux médecins généralistes,APL aux médecins généralistes de moins de 65 ans,Population standardisée 2013 pour la médecine générale,Population totale 2013,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001.0,L' Abergement-Clémenciat,2.382,2.21,763.384,767,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002.0,L' Abergement-de-Varey,2.763,2.673,235.303,236,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004.0,Ambérieu-en-Bugey,4.42,4.324,14176.035,14359,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005.0,Ambérieux-en-Dombes,4.24,4.24,1605.501,1635,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006.0,Ambléon,1.103,0.926,111.035,108,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes


In [42]:
apl_2021.tail()

,Code commune INSEE,Commune,APL aux médecins généralistes,APL aux médecins généralistes de moins de 65 ans,Population standardisée 2019 pour la médecine générale,Population totale 2019,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
187975,75116.0,Paris 16e Arrondissement,3.546,2.768,167614.21,165523,PARIS 16,75116,PARIS,NaN,48.860399,2.262100,116.0,NaN,Paris 16,Paris 16,75,Paris,11.0,Île-de-France
187976,75117.0,Paris 17e Arrondissement,4.662,3.752,159595.107,166543,PARIS 17,75017,PARIS,NaN,48.887337,2.307486,117.0,NaN,Paris 17,Paris 17,75,Paris,11.0,Île-de-France
187977,75118.0,Paris 18e Arrondissement,4.267,3.479,180410.01,192468,PARIS 18,75018,PARIS,NaN,48.892735,2.348712,118.0,NaN,Paris 18,Paris 18,75,Paris,11.0,Île-de-France
187978,75119.0,Paris 19e Arrondissement,4.672,3.843,176730.213,184573,PARIS 19,75019,PARIS,NaN,48.886869,2.384694,119.0,NaN,Paris 19,Paris 19,75,Paris,11.0,Île-de-France
187979,75120.0,Paris 20e Arrondissement,4.17,3.404,187719.246,194994,PARIS 20,75020,PARIS,NaN,48.863187,2.400820,120.0,NaN,Paris 20,Paris 20,75,Paris,11.0,Île-de-France


#### create a grouped by departement and calculate apl weight

In [43]:
def group_by_departement(apl, year):
    # Calculate the weighted APL for each commune
    apl['Weighted APL'] = apl['APL aux médecins généralistes'] * apl['Population standardisée '+year+' pour la médecine générale']
    
    # Group by department and calculate the weighted average APL
    apl = apl.groupby('nom_departement').agg({
        'Weighted APL': 'sum',
        'Population standardisée '+year+' pour la médecine générale': 'sum'
    }).reset_index()
    
    apl['Weighted Average APL'] = apl['Weighted APL'] / apl['Population standardisée '+year+' pour la médecine générale']
    return apl

In [44]:
grouped_apl_2015 = group_by_departement(apl_2015, '2013')
grouped_apl_2016 = group_by_departement(apl_2016, '2014')
grouped_apl_2017 = group_by_departement(apl_2017, '2015')
grouped_apl_2018 = group_by_departement(apl_2018, '2016')
grouped_apl_2019 = group_by_departement(apl_2019, '2017')
grouped_apl_2021 = group_by_departement(apl_2021, '2019')

In [45]:
grouped_apl_2021.head()

,nom_departement,Weighted APL,Population standardisée 2019 pour la médecine générale,Weighted Average APL
0,Ain,2324181.109843,821375.861,2.829619
1,Aisne,1946143.935294,602127.136,3.232115
2,Allier,1135747.349915,360578.798,3.14979
3,Alpes-Maritimes,17294652.315639,3559522.463,4.8587
4,Alpes-de-Haute-Provence,869861.721134,221412.374,3.928695


In [46]:
grouped_apl_2018.tail()

,nom_departement,Weighted APL,Population standardisée 2016 pour la médecine générale,Weighted Average APL
95,Vendée,2757736.243661,842973.148,3.27144
96,Vienne,2258691.939296,521025.383,4.33509
97,Vosges,1673065.674088,426198.318,3.925557
98,Yonne,1399354.625015,510436.051,2.741489
99,Yvelines,4986207.116786,1657017.94,3.009145


#### France Regions

In [47]:
# Load the GeoJSON file with France's regions
france_regions = gpd.read_file("https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/france.geojson")

In [48]:
france_regions = france_regions.rename(columns={'nom': 'nom_departement'})

In [49]:
france_regions.tail()

,code,nom_departement,geometry
91,91,Essonne,"POLYGON ((2.22656 48.77610, 2.23298 48.76620, ..."
92,92,Hauts-de-Seine,"POLYGON ((2.29097 48.95097, 2.32697 48.94536, ..."
93,93,Seine-Saint-Denis,"POLYGON ((2.55306 49.00982, 2.58031 48.99159, ..."
94,94,Val-de-Marne,"POLYGON ((2.33190 48.81701, 2.36395 48.81632, ..."
95,95,Val-d'Oise,"POLYGON ((2.59052 49.07965, 2.57203 49.06149, ..."


#### Merge to get departement cordinate

In [50]:
def merge_to_get_cordinate(apl):
    return france_regions.merge(apl, on='nom_departement')

In [51]:
departement_apl_2015 = merge_to_get_cordinate(grouped_apl_2015)
departement_apl_2016 = merge_to_get_cordinate(grouped_apl_2016)
departement_apl_2017 = merge_to_get_cordinate(grouped_apl_2017)
departement_apl_2018 = merge_to_get_cordinate(grouped_apl_2018)
departement_apl_2019 = merge_to_get_cordinate(grouped_apl_2019)
departement_apl_2021 = merge_to_get_cordinate(grouped_apl_2021)

In [52]:
departement_apl_2021.head()

,code,nom_departement,geometry,Weighted APL,Population standardisée 2019 pour la médecine générale,Weighted Average APL
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",2324181.109843,821375.861,2.829619
1,02,Aisne,"POLYGON ((4.04797 49.40564, 4.03991 49.39740, ...",1946143.935294,602127.136,3.232115
2,03,Allier,"POLYGON ((3.03207 46.79491, 3.04907 46.75808, ...",1135747.349915,360578.798,3.14979
3,04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.69209 44.18648, ...",869861.721134,221412.374,3.928695
4,05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.29922 45.10855, ...",1037283.608615,204665.231,5.068197


In [53]:
departement_apl_2016.head()

,code,nom_departement,geometry,Weighted APL,Population standardisée 2014 pour la médecine générale,Weighted Average APL
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",2497946.186231,799376.137,3.12487
1,02,Aisne,"POLYGON ((4.04797 49.40564, 4.03991 49.39740, ...",2171969.337465,611893.447,3.549588
2,03,Allier,"POLYGON ((3.03207 46.79491, 3.04907 46.75808, ...",1267466.68451,366430.643,3.458954
3,04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.69209 44.18648, ...",920209.020764,217388.438,4.233017
4,05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.29922 45.10855, ...",1018688.52497,200835.3,5.072258


#### Maps

In [54]:
def notebook_map(apl):
    # Create a folium map centered at a specific location
    m = folium.Map(location=[48, 2], zoom_start=6)
    
    # Add a choropleth layer to the map
    Choropleth(
        geo_data=apl,
        data=apl,
        columns=['code', 'Weighted Average APL'],
        key_on='feature.properties.code',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Weighted Average APL'
    ).add_to(m)
    
    # Create markers for each region with tooltips
    for idx, row in apl.iterrows():
        folium.Marker(
            location=row['geometry'].centroid.coords[0][::-1],  # Use the centroid of the region
            popup=f"Department: {row['nom_departement']}<br>APL: {row['Weighted Average APL']:.2f}",
        ).add_to(m)
    
    return m


In [55]:
m_2015 = notebook_map(departement_apl_2015)
m_2016 = notebook_map(departement_apl_2016)
m_2017 = notebook_map(departement_apl_2017)
m_2018 = notebook_map(departement_apl_2018)
m_2019 = notebook_map(departement_apl_2019)

#### Interactive years

In [57]:
import ipywidgets
from IPython.display import display, HTML

# Define the maps for different years
maps = {
    "2015": m_2015,
    "2016": m_2016,
    "2017": m_2017,
    "2018": m_2018,
    "2019": m_2019,
}

# Create a dropdown widget to select the year
year_selector = ipywidgets.Dropdown(
    options=list(maps.keys()),
    value="2015",
    description="Select a Year:",
)

# Create an output widget to display the selected map
map_output = ipywidgets.Output()

# Function to update the displayed map based on the selected year
def update_map(change):
    selected_year = year_selector.value
    selected_map = maps[selected_year]

    map_output.clear_output()
    with map_output:
        display(selected_map)

# Attach the update_map function to the year_selector widget
year_selector.observe(update_map, names='value')

# Display the widgets
display(year_selector)
display(map_output)


Dropdown(description='Select a Year:', options=('2015', '2016', '2017', '2018', '2019'), value='2015')

Output()